In [40]:
import numpy as np
import torch
from tqdm.notebook import tqdm
from transformers import AutoTokenizer, pipeline

import pandas as pd

import ipywidgets as widgets
from IPython.display import display, clear_output

import plotly.express as px

import matplotlib.pyplot as plt

import seaborn as sns

from collections import Counter
import re
import nltk
from nltk.corpus import stopwords

from nltk.util import ngrams


import plotly.graph_objects as go
import pandas as pd

In [41]:

!pip install ipywidgets

In [31]:
#df_combined = pd.read_csv("../aggregated_data/combined_review_data.csv")
df_combined = pd.read_csv("combined_review_data.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'combined_review_data.csv'

In [ ]:
df_combined.head()

,review,review_datetime,data_source,app_name,upvote_count,total_comments,app_rating
0,uber eats for owls? will they ever come out wi...,2025-04-20 21:51:15,Reddit,UberEats,1.0,2.0,NaN
1,serious question yall is it worth going out to...,2025-04-20 21:41:21,Reddit,UberEats,1.0,1.0,NaN
2,ubereats charged me for a successful chargebac...,2025-04-20 20:50:04,Reddit,UberEats,1.0,2.0,NaN
3,ubereats driver scammed me by buying half the ...,2025-04-20 20:48:13,Reddit,UberEats,1.0,9.0,NaN
4,ubereats why you do this? family went out of t...,2025-04-20 20:19:15,Reddit,UberEats,1.0,3.0,NaN


In [ ]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126725 entries, 0 to 126724
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   review           126716 non-null  object 
 1   review_datetime  126725 non-null  object 
 2   data_source      126725 non-null  object 
 3   app_name         126725 non-null  object 
 4   upvote_count     126290 non-null  float64
 5   total_comments   21157 non-null   float64
 6   app_rating       105133 non-null  float64
dtypes: float64(3), object(4)
memory usage: 6.8+ MB


In [ ]:
df_combined.shape

(126725, 7)

## Sentiment Analysis

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

In [ ]:

sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment",
    device=0 if torch.cuda.is_available() else -1  # Use GPU if available
)

Device set to use cuda:0


In [ ]:
label_map = {
    "LABEL_0": "Negative",
    "LABEL_1": "Neutral",
    "LABEL_2": "Positive"
}


In [ ]:
def safe_sentiment(text):
    if pd.isna(text) or text.strip() == "":
        return "Neutral"
    try:
        # Truncate input text at tokenizer level
        result = sentiment_analyzer(
            text,
            truncation=True,
            max_length=512
        )[0]
        return label_map.get(result['label'], "UNKNOWN")
    except Exception as e:
        print(f"Error processing text: {e}")
        return "ERROR"

In [ ]:
df_combined['sentiment'] = df_combined['review'].apply(safe_sentiment)

In [ ]:
df_combined['sentiment'].value_counts()

,count
sentiment,
Negative,62305
Positive,45037
Neutral,19383


In [42]:

# df_combined.to_csv("../aggregated_data/combined_review_data_with_sentiment.csv", index=False)
df_combined.to_csv("combined_review_data_with_sentiment.csv", index=False)

In [43]:


df_combined = pd.read_csv('combined_review_data_with_sentiment.csv')

In [44]:
df_combined.shape

(126725, 8)

In [46]:


def plot_sentiment_distribution(selected_app):
    # Filter your dataframe based on app
    filtered_df = df[df['app_name'] == selected_app]

    # Plotting
    sentiment_counts = filtered_df['sentiment'].value_counts()
    sentiment_counts.plot(kind='bar')

    plt.title(f'Sentiment Distribution for {selected_app}')
    plt.xlabel('Sentiment')
    plt.ylabel('Count')
    plt.grid(True)
    plt.show()


In [47]:
!pip install dash pyngrok


In [48]:


# --------------------------------------
# Clean data
# --------------------------------------
df_combined['app_name'] = df_combined['app_name'].str.strip().str.title().replace({
    'Ubereats': 'UberEats',
    'Doordash': 'DoorDash',
    'Grubhub': 'GrubHub'
})
df_combined['sentiment'] = df_combined['sentiment'].str.lower()

# --------------------------------------
# App colors
# --------------------------------------
app_colors = {
    'UberEats': {
        'neutral': '#90EE90', 'positive': '#32CD32', 'negative': '#006400',
    },
    'DoorDash': {
        'neutral': '#FFB6C1', 'positive': '#FF6347', 'negative': '#8B0000',
    },
    'GrubHub': {
        'neutral': '#FFDAB9', 'positive': '#FFA500', 'negative': '#FF8C00',
    }
}

# --------------------------------------
# Prepare data for all apps
# --------------------------------------
apps = ['UberEats', 'DoorDash', 'GrubHub']
fig = go.Figure()
buttons = []

for i, app in enumerate(apps):
    app_df = df_combined[df_combined['app_name'] == app]
    sentiment_counts = app_df['sentiment'].value_counts()
    sentiments = ['neutral', 'positive', 'negative']
    values = [sentiment_counts.get(s, 0) for s in sentiments]
    colors = [app_colors[app][s] for s in sentiments]

    # Add trace (but only show the first one)
    fig.add_trace(go.Pie(
        labels=sentiments,
        values=values,
        hole=0.5,
        marker=dict(colors=colors),
        name=app,
        visible=(i == 0),  # Only the first is visible initially
        textinfo='percent+label',
        insidetextorientation='radial'
    ))

    # Add dropdown button for this trace
    button = dict(label=app,
                  method="update",
                  args=[{"visible": [j == i for j in range(len(apps))]},
                        {"title": f"Sentiment Distribution for {app}"}])
    buttons.append(button)

# --------------------------------------
# Add dropdown to figure
# --------------------------------------
fig.update_layout(
    updatemenus=[dict(
        type="dropdown",
        direction="down",
        buttons=buttons,
        x=0.1,
        y=1.15
    )],
    title="Sentiment Distribution for UberEats",  # Initial
    height=500
)

fig.show()


# The donut chart above illustrates the sentiment distribution of customer reviews for food delivery apps. Each segment of the chart represents the percentage of reviews that are classified as Positive, Neutral, or Negative. This visualization offers a clear and comparative view of user satisfaction levels, highlighting how customer perception varies across different apps.

In [52]:


# ---------------------------
# Clean your data
# ---------------------------
df_combined['app_name'] = df_combined['app_name'].str.strip().str.title().replace({
    'Ubereats': 'UberEats',
    'Doordash': 'DoorDash',
    'Grubhub': 'GrubHub'
})
df_combined['sentiment'] = df_combined['sentiment'].str.lower()
df_combined['data_source'] = df_combined['data_source'].str.strip().str.title()

# ---------------------------
# Define app colors
# ---------------------------
app_colors = {
    'UberEats': {
        'neutral': '#90EE90', 'positive': '#32CD32', 'negative': '#006400',
    }
}

# ---------------------------
# Set fixed app, vary data_source
# ---------------------------
selected_app = 'UberEats'
sources = df_combined['data_source'].unique()
fig = go.Figure()
buttons = []

# Build a donut for each source
for i, src in enumerate(sources):
    app_df = df_combined[
        (df_combined['app_name'] == selected_app) &
        (df_combined['data_source'] == src)
    ]
    sentiments = ['neutral', 'positive', 'negative']
    counts = app_df['sentiment'].value_counts()
    values = [counts.get(s, 0) for s in sentiments]
    colors = [app_colors[selected_app][s] for s in sentiments]

    # Add trace (only show first initially)
    fig.add_trace(go.Pie(
        labels=sentiments,
        values=values,
        hole=0.5,
        marker=dict(colors=colors),
        name=src,
        visible=(i == 0),
        textinfo='percent+label',
        insidetextorientation='radial'
    ))

    buttons.append(dict(
        label=src,
        method="update",
        args=[{"visible": [j == i for j in range(len(sources))]},
              {"title": f"Sentiment Distribution for {src}"}]
    ))

# ---------------------------
# Add dropdown menu
# ---------------------------
fig.update_layout(
    updatemenus=[dict(
        type="dropdown",
        buttons=buttons,
        direction="down",
        x=0.1,
        y=1.15
    )],
    title=f"Sentiment Distribution for {sources[0]}",
    height=500
)

fig.show()


# The donut chart visualizes the sentiment distribution for a selected app across various review platforms.Each segment of the chart represents the percentage of reviews labeled as Positive, Neutral, or Negative. This allows for comparison of user sentiment trends across different communities and platforms where the app is reviewed.

In [50]:

# -----------------------------
# Prepare data
# -----------------------------
reddit_reviews = df_combined[df_combined['data_source'] == 'Reddit'].copy()
reddit_reviews['sentiment'] = reddit_reviews['sentiment'].str.capitalize()
reddit_reviews['review'] = reddit_reviews['review'].astype(str)

# Define colors
sentiment_colors = {
    'Positive': 'green',
    'Negative': 'red',
    'Neutral': 'blue'
}

# Apps to show
apps = ['UberEats', 'DoorDash', 'GrubHub']

# Build the figure
fig = go.Figure()
buttons = []

for i, app in enumerate(apps):
    app_df = reddit_reviews[reddit_reviews['app_name'] == app]
    top_10 = app_df.sort_values(by='upvote_count', ascending=False).head(10)
    if top_10.empty:
        continue

    # Shorten long reviews
    review_labels = top_10['review'].apply(lambda x: x[:80] + '...' if len(x) > 80 else x)
    bar_colors = top_10['sentiment'].map(sentiment_colors).fillna('gray')

    fig.add_trace(go.Bar(
        x=top_10['upvote_count'],
        y=review_labels,
        orientation='h',
        marker_color=bar_colors,
        name=app,
        visible=(i == 0),
        hovertext=top_10['review'],
    ))

    # Dropdown button to toggle visibility
    buttons.append(dict(
        label=app,
        method="update",
        args=[
            {"visible": [j == i for j in range(len(apps))]},
            {"title": f"Top 10 Upvoted Reviews for {app}"}
        ]
    ))

# Final layout
fig.update_layout(
    title=f"Top 10 Upvoted Reviews for {apps[0]} ",
    xaxis_title="Upvote Count",
    yaxis_title="Review",
    yaxis=dict(autorange='reversed'),
    height=600,
    updatemenus=[dict(
        type="dropdown",
        buttons=buttons,
        x=0.1,
        y=1.15
    )]
)

fig.show()


# This horizontal bar chart displays the top 10 most upvoted customer reviews for a selected food delivery app. Each bar is color-coded by sentiment—blue for Neutral, red for Negative, and green (if present) for Positive—providing a quick visual reference to the tone of each review. The dropdown allows switching between UberEats, DoorDash, and GrubHub for comparison.

In [53]:


# -----------------------------
# Clean data
# -----------------------------
df_combined['app_name'] = df_combined['app_name'].str.strip().str.title().replace({
    'Ubereats': 'UberEats',
    'Doordash': 'DoorDash',
    'Grubhub': 'GrubHub'
})
df_combined['sentiment'] = df_combined['sentiment'].str.capitalize()
df_combined['review'] = df_combined['review'].astype(str)
df_combined['data_source'] = df_combined['data_source'].str.strip().str.title()

# -----------------------------
# Define sentiment colors
# -----------------------------
sentiment_colors = {
    'Positive': 'green',
    'Negative': 'red',
    'Neutral': 'blue'
}

# -----------------------------
# Set app and get sources
# -----------------------------
selected_app = 'UberEats'
sources = sorted(df_combined['data_source'].unique())

# -----------------------------
# Build Plotly figure
# -----------------------------
fig = go.Figure()
buttons = []

for i, source in enumerate(sources):
    source_df = df_combined[
        (df_combined['data_source'] == source) &
        (df_combined['app_name'] == selected_app)
    ]
    top_10 = source_df.sort_values(by='upvote_count', ascending=False).head(10)
    if top_10.empty:
        continue

    review_labels = top_10['review'].apply(lambda x: x[:80] + '...' if len(x) > 80 else x)
    bar_colors = top_10['sentiment'].map(sentiment_colors).fillna('gray')

    fig.add_trace(go.Bar(
        x=top_10['upvote_count'],
        y=review_labels,
        orientation='h',
        marker_color=bar_colors,
        name=source,
        visible=(i == 0),
        hovertext=top_10['review'],
    ))

    buttons.append(dict(
        label=source,
        method="update",
        args=[
            {"visible": [j == i for j in range(len(sources))]},
            {"title": f"Top 10 Upvoted Reviews for  {source}"}
        ]
    ))

# -----------------------------
# Final layout
# -----------------------------
fig.update_layout(
    title=f"Top 10 Upvoted Reviews for  {sources[0]}",
    xaxis_title="Upvote Count",
    yaxis_title="Review",
    yaxis=dict(autorange='reversed'),
    height=600,
    updatemenus=[dict(
        type="dropdown",
        buttons=buttons,
        x=0.1,
        y=1.15
    )]
)

fig.show()


# This horizontal bar chart displays the top 10 most upvoted customer reviews from the selected data source. Users can switch between platforms such as Google Play, App Store, and Reddit using the dropdown. Each bar represents a highly upvoted review and is color-coded by sentiment — red for Negative, blue for Neutral, and green for Positive. This plot highlights not only the most talked-about experiences on each platform but also the general sentiment users express toward food delivery apps depending on where the review was posted.